# Modelos Baseline

En este notebook se implementaran los modelos baseline del proyecto y se guardaran las métricas con el mismo dataset que se utilizará para el modelo principal para hacer bentchmarking

## Lectura de los datos

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os
import zipfile
import requests
from io import BytesIO
from tqdm import tqdm

In [ ]:
"""
data_dir = "../data/raw"
os.makedirs(data_dir, exist_ok=True)

url = "http://files.grouplens.org/datasets/movielens/ml-25m.zip"
response = requests.get(url)
with zipfile.ZipFile(BytesIO(response.content)) as z:
    z.extractall(os.path.join(data_dir, "ml-25m"))
"""

In [3]:
ratings_csv = pd.read_csv("../data/raw/ml-25m/ratings.csv")
movies_csv = pd.read_csv("../data/raw/ml-25m/movies.csv")

In [4]:
ratings_csv

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [5]:
movies_csv

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


## Separamos train/test/val

Los splits seran de tamaño 80, 10, 10

In [1]:
ratings_train, ratings_test, ratings_val = np.split(ratings_csv.sample(frac=1, random_state=42),
                                                    [int(.8*len(ratings_csv)), int(.9*len(ratings_csv))])

NameError: name 'np' is not defined

## Random model

Como que no hace falta ningun tratamiento de datos extra, procedemos ya